In [1]:
import pandas as pd
import numpy as np
import re
import nltk
pd.set_option('display.max_columns',None)

In [2]:
df = pd.read_csv("IMDB_Top250Engmovies2_OMDB_Detailed.csv")
df.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [3]:
df['Plot'][0]

'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.'

## Data Preprocessing

In [4]:
# convert lowercase and remove numbers, punctuation, spaces, etc.,
df['clean_plot'] = df['Plot'].str.lower()
df['clean_plot'] = df['clean_plot'].apply(lambda x: re.sub('^[a-zA-a]',' ',x))
df['clean_plot'] = df['clean_plot'].apply(lambda x: re.sub('\s+',' ',x))
df['clean_plot']

0       wo imprisoned men bond over a number of years...
1       he aging patriarch of an organized crime dyna...
2       he early life and career of vito corleone in ...
3       hen the menace known as the joker emerges fro...
4       jury holdout attempts to prevent a miscarriag...
                             ...                        
245     he desperate life of a chronic alcoholic is f...
246     20-something supervising staff member of a re...
247     newspaper editor uses every trick in the book...
248     n old man makes a long journey by lawn-mover ...
249     mumbai teen reflects on his upbringing in the...
Name: clean_plot, Length: 250, dtype: object

In [5]:
nltk.download('punkt')
df['clean_plot'] = df['clean_plot'].fillna('')  # Replace NaN with an empty string
df['clean_plot'] = df['clean_plot'].astype(str)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# tekenize the sentence
df['clean_plot'] = df['clean_plot'].apply(lambda x: nltk.word_tokenize(x))
df['clean_plot']

0      [wo, imprisoned, men, bond, over, a, number, o...
1      [he, aging, patriarch, of, an, organized, crim...
2      [he, early, life, and, career, of, vito, corle...
3      [hen, the, menace, known, as, the, joker, emer...
4      [jury, holdout, attempts, to, prevent, a, misc...
                             ...                        
245    [he, desperate, life, of, a, chronic, alcoholi...
246    [20-something, supervising, staff, member, of,...
247    [newspaper, editor, uses, every, trick, in, th...
248    [n, old, man, makes, a, long, journey, by, law...
249    [mumbai, teen, reflects, on, his, upbringing, ...
Name: clean_plot, Length: 250, dtype: object

In [7]:
# Remove stopwords
stop_words = nltk.corpus.stopwords.words('english')
plot = []

for sentence in df['clean_plot']:
    temp = []
    for word in sentence:
        if word not in stop_words or len(word) >= 3:  
            temp.append(word)
    plot.append(temp)  

plot


[['wo',
  'imprisoned',
  'men',
  'bond',
  'over',
  'number',
  'years',
  ',',
  'finding',
  'solace',
  'and',
  'eventual',
  'redemption',
  'through',
  'acts',
  'common',
  'decency',
  '.'],
 ['aging',
  'patriarch',
  'organized',
  'crime',
  'dynasty',
  'transfers',
  'control',
  'his',
  'clandestine',
  'empire',
  'his',
  'reluctant',
  'son',
  '.'],
 ['early',
  'life',
  'and',
  'career',
  'vito',
  'corleone',
  '1920s',
  'new',
  'york',
  'portrayed',
  'while',
  'his',
  'son',
  ',',
  'michael',
  ',',
  'expands',
  'and',
  'tightens',
  'his',
  'grip',
  'the',
  'family',
  'crime',
  'syndicate',
  '.'],
 ['hen',
  'the',
  'menace',
  'known',
  'the',
  'joker',
  'emerges',
  'from',
  'his',
  'mysterious',
  'past',
  ',',
  'wreaks',
  'havoc',
  'and',
  'chaos',
  'the',
  'people',
  'gotham',
  ',',
  'the',
  'dark',
  'knight',
  'must',
  'accept',
  'one',
  'the',
  'greatest',
  'psychological',
  'and',
  'physical',
  'tests',
 

In [8]:
df['clean_plot']

0      [wo, imprisoned, men, bond, over, a, number, o...
1      [he, aging, patriarch, of, an, organized, crim...
2      [he, early, life, and, career, of, vito, corle...
3      [hen, the, menace, known, as, the, joker, emer...
4      [jury, holdout, attempts, to, prevent, a, misc...
                             ...                        
245    [he, desperate, life, of, a, chronic, alcoholi...
246    [20-something, supervising, staff, member, of,...
247    [newspaper, editor, uses, every, trick, in, th...
248    [n, old, man, makes, a, long, journey, by, law...
249    [mumbai, teen, reflects, on, his, upbringing, ...
Name: clean_plot, Length: 250, dtype: object

In [9]:
df.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response,clean_plot
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True,"[wo, imprisoned, men, bond, over, a, number, o..."
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True,"[he, aging, patriarch, of, an, organized, crim..."
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True,"[he, early, life, and, career, of, vito, corle..."
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True,"[hen, the, menace, known, as, the, joker, emer..."
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True,"[jury, holdout, attempts, to, prevent, a, misc..."


In [10]:
df['Genre'] = df['Genre'].apply(lambda x: x.split(','))
df['Actors'] = df['Actors'].apply(lambda x: x.split(',')[:4])
df['Director'] = df['Director'].apply(lambda x: x.split(','))

In [11]:
df['Actors'][0]

['Tim Robbins', ' Morgan Freeman', ' Bob Gunton', ' William Sadler']

In [20]:
def clean(sentence):
    temp = []
    for word in sentence:
        temp.append(word.lower().replace(' ',''))
        return temp

In [21]:
df['Genre'] = [clean(x) for x in df['Genre']]
df['Actors'] = [clean(x) for x in df['Actors']]
df['Director'] = [clean(x) for x in df['Director']]

In [22]:
df['Actors'][0]

['timrobbins']

In [ ]:
# combining all the columns data
columns = ['clean_plot', 'Genre', 'Actors', 'Director']
l = []
for i in range(len(df)):
    words = ''
    for col in columns:
        words += ' '.join(df[col][i]) + ' '
    l.append(words)
l

In [24]:
df['clean_input'] = l
df = df[['Title', 'clean_input']]
df.head()

,Title,clean_input
0,The Shawshank Redemption,wo imprisoned men bond over a number of years ...
1,The Godfather,he aging patriarch of an organized crime dynas...
2,The Godfather: Part II,he early life and career of vito corleone in 1...
3,The Dark Knight,hen the menace known as the joker emerges from...
4,12 Angry Men,jury holdout attempts to prevent a miscarriage...


## Feature Extraction

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf = TfidfVectorizer()
feature = tfidf.fit_transform(df['clean_input'])

In [28]:
# create cosinge similarity matrix
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(feature, feature)
print(cosine_sim)

[[1.         0.0552573  0.0467397  ... 0.         0.         0.02058261]
 [0.0552573  1.         0.20096603 ... 0.02206497 0.03966878 0.05611053]
 [0.0467397  0.20096603 1.         ... 0.02499527 0.01405441 0.08341722]
 ...
 [0.         0.02206497 0.02499527 ... 1.         0.01387407 0.02805565]
 [0.         0.03966878 0.01405441 ... 0.01387407 1.         0.01238343]
 [0.02058261 0.05611053 0.08341722 ... 0.02805565 0.01238343 1.        ]]


## Movie Recommendation

In [29]:
index = pd.Series(df['Title'])
index.head()

0    The Shawshank Redemption
1               The Godfather
2      The Godfather: Part II
3             The Dark Knight
4                12 Angry Men
Name: Title, dtype: object

In [55]:
def recommend_movies(title):
    movies = []
    try:
        idx = df[df['Title'] == title].index[0]
    except IndexError:
        print(f"Movie '{title}' not found in the dataset.")
        return []
    print(idx)
    score = pd.Series(cosine_sim[idx]).sort_values(ascending=False)
    top10 = list(score.iloc[1:11].index)
    print(top10)
    for i in top10:
         movies.append(df['Title'][i])
    
    return movies

In [56]:
recommend_movies('The Dark Knight Rises')

51
[3, 89, 9, 17, 59, 13, 199, 156, 224, 31]


['The Dark Knight',
 'Batman Begins',
 'The Lord of the Rings: The Fellowship of the Ring',
 'Star Wars: Episode IV - A New Hope',
 'Star Wars: Episode VI - Return of the Jedi',
 'The Lord of the Rings: The Two Towers',
 'Guardians of the Galaxy',
 'The Grapes of Wrath',
 'Rio Bravo',
 'Raiders of the Lost Ark']

In [57]:
index[index == 'The Dark Knight Rises'].index[0]

51

In [58]:
pd.Series(cosine_sim[51]).sort_values(ascending=False)

51     1.000000
3      0.316514
89     0.224949
9      0.153330
17     0.146803
         ...   
218    0.000000
216    0.000000
65     0.000000
110    0.000000
62     0.000000
Length: 250, dtype: float64

In [59]:
recommend_movies('Avatar')

Movie 'Avatar' not found in the dataset.


[]

In [60]:
recommend_movies('The Avengers')

219
[203, 199, 6, 3, 25, 55, 222, 136, 89, 190]


['Guardians of the Galaxy Vol. 2',
 'Guardians of the Galaxy',
 'The Lord of the Rings: The Return of the King',
 'The Dark Knight',
 'Interstellar',
 'Aliens',
 'The Martian',
 'Kill Bill: Vol. 1',
 'Batman Begins',
 'The Terminator']

In [61]:
len(df)

250

In [62]:
recommend_movies('The Shawshank Redemption')

0
[7, 15, 226, 29, 54, 191, 95, 148, 234, 18]


['Pulp Fiction',
 'Goodfellas',
 'Rope',
 'The Green Mile',
 'Once Upon a Time in America',
 'Groundhog Day',
 'The Great Escape',
 "Hachi: A Dog's Tale",
 'Blood Diamond',
 'Se7en']